In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import argparse
import glob
import cmlreaders as cml
import json
from matplotlib.ticker import FuncFormatter
import warnings
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
from scipy import stats
data_dir = "/scratch/new_courier_pilot/"
subject='R1620J'
exp = 'EFRCourierOpenLoop'
session= 3
reader = cml.CMLReader(subject, exp,3)
evs = reader.load('task_events')


In [101]:
evs['inside_stimuli'] = -999
stimuli_indices = evs[evs['type'] == 'STIM'].index

for i in stimuli_indices:
    current_offset = evs.loc[i, 'eegoffset']
    evs.loc[i, 'inside_stimuli'] = evs.loc[i, 'stim_params']['burst_freq']
    # Keep checking subsequent rows until difference >= 3000
    j = i+1
    rows_to_assign = []  # Store indices of rows to assign

    while j < len(evs):
        next_offset = evs.loc[j, 'eegoffset']
        time_diff = abs(next_offset - current_offset)
        #print(time_diff)
        if time_diff < 3000:
            # Still within 3 seconds, add this row to assignment list
            rows_to_assign.append(j)
            j += 1
        else:
            # Found a row with difference >= 4000, stop checking
            break

    # Assign burst_freq to all rows that were within 4000ms of the STIM
    for row_idx in rows_to_assign:
        evs.loc[row_idx, 'inside_stimuli'] = evs.loc[i, 'stim_params']['burst_freq']
        evs.loc[i, 'inside_stimuli'] = evs.loc[i, 'stim_params']['burst_freq']
# Get unique trials for this session
trials = evs['trial'].unique()
# Find indices of all WORD events
word_indices = evs[evs['type'] == "WORD"].index

# Get the events immediately before WORD events
preceding_events = evs.iloc[word_indices - 1]

# Find which preceding events are STIM type
stim_mask = preceding_events['type'] == 'STIM'
stim_indices = preceding_events[stim_mask].index

# Update the inside_stimuli column for WORD events that follow STIM events
for stim_idx in stim_indices:
    word_idx = stim_idx + 1
    burst_freq = evs.loc[stim_idx, 'stim_params']['burst_freq']
    evs.loc[word_idx, 'inside_stimuli'] = burst_freq
    evs.loc[stim_idx, 'inside_stimuli'] = burst_freq

In [103]:

FirstTrial=evs[evs['trial']==1]
pd.set_option('display.max_rows', None)
FirstTrial[['store','type']]
Events=evs[evs['type']=='pointing finished']
Events=Events.reset_index(drop=True)
Words=evs[evs['type']=='WORD']
Words=Words.reset_index(drop=True)
Words['new_time']=Events['mstime']
Words['inside_stimuli']
Words

,eegoffset,correctPointingDirection,eegfile,efr_mark,experiment,finalrecalled,intruded,intrusion,item,itemno,...,stim_params,store,storeX,storeZ,subject,submittedPointingDirection,trial,type,inside_stimuli,new_time
0,171803,-999.0,R1620J_EFRCourierOpenLoop_3_22Dec23_2252,-999,EFRCourierOpenLoop,0,0,-999,ASPIRIN,-999,...,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",pharmacy,-3.070312,-58.500000,R1620J,-999.0,0,WORD,3,1654633375710
1,191815,-999.0,R1620J_EFRCourierOpenLoop_3_22Dec23_2252,-999,EFRCourierOpenLoop,0,0,-999,FLUORIDE,-999,...,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",dentist,-29.562500,-37.250000,R1620J,-999.0,0,WORD,-999,1654633404583
2,235867,-999.0,R1620J_EFRCourierOpenLoop_3_22Dec23_2252,-999,EFRCourierOpenLoop,0,0,-999,AMPLIFIER,-999,...,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",music_store,32.187500,-60.906250,R1620J,-999.0,0,WORD,-999,1654633443204
3,259491,-999.0,R1620J_EFRCourierOpenLoop_3_22Dec23_2252,-999,EFRCourierOpenLoop,0,1,-999,EMERALD,-999,...,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",jewelry_store,59.281250,-9.156250,R1620J,-999.0,0,WORD,-999,1654633468645
4,283892,-999.0,R1620J_EFRCourierOpenLoop_3_22Dec23_2252,-999,EFRCourierOpenLoop,0,0,-999,HAM,-999,...,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",grocery_store,-9.093750,17.859375,R1620J,-999.0,0,WORD,3,1654633494670
5,314434,-999.0,R1620J_EFRCourierOpenLoop_3_22Dec23_2252,-999,EFRCourierOpenLoop,0,0,-999,FERNS,-999,...,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",florist,-38.937500,-61.875000,R1620J,-999.0,0,WORD,-999,1654633522072
6,339249,-999.0,R1620J_EFRCourierOpenLoop_3_22Dec23_2252,-999,EFRCourierOpenLoop,0,0,-999,COFFEE,-999,...,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",cafe,14.492188,-33.750000,R1620J,-999.0,0,WORD,-999,1654633549615
7,366092,-999.0,R1620J_EFRCourierOpenLoop_3_22Dec23_2252,-999,EFRCourierOpenLoop,0,0,-999,GARLIC,-999,...,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",pizzeria,35.406250,-33.125000,R1620J,-999.0,0,WORD,-999,1654633570027
8,382293,-999.0,R1620J_EFRCourierOpenLoop_3_22Dec23_2252,-999,EFRCourierOpenLoop,0,0,-999,ROPE,-999,...,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",gym,62.343750,-33.500000,R1620J,-999.0,0,WORD,3,1654633596671
9,405894,-999.0,R1620J_EFRCourierOpenLoop_3_22Dec23_2252,-999,EFRCourierOpenLoop,0,0,-999,PUMP,-999,...,"{'_remove': 1, 'amplitude': -1.0, 'anode_label...",bike_shop,35.062500,26.656250,R1620J,-999.0,0,WORD,-999,1654633614072


In [114]:
# Select the columns (assuming Words is a DataFrame)


# Get unique stores (add parentheses to call the method)
allstore = Words['store'].unique()

# Initialize empty list to collect results
result_list = []

for store in allstore:
    # Filter data for current store (add missing closing bracket)
    new_words = Words[Words['store'] == store]
    
    # Loop through rows (use range(len()) for iteration)
    for i in range(len(new_words)):
        result = {
            'store': new_words.iloc[i]['store'],
            # Fix time calculation (this gives 0 - did you mean something else?)
            'time': new_words.iloc[i]['mstime'] - new_words.iloc[i]['new_time'],  
            # Fix column reference (assuming you meant a column from new_words)
            'condition': new_words.iloc[i]['inside_stimuli'],  # or whatever column you need
            'visit_time': i,
            'trial':new_words.iloc[i]['trial'],
            'session':new_words.iloc[i]['session'],
            'session':new_words.iloc[i]['subject']
        }
        result_list.append(result)

# Convert to DataFrame at the end (more efficient than repeated append)
result_pd = pd.DataFrame(result_list)
    

In [106]:
new_words.iloc[i]['mstime']

1654636077735

In [115]:
result_pd


,store,time,condition,visit_time,trial,session
0,pharmacy,20471,3,0,0,R1620J
1,pharmacy,41602,8,1,1,R1620J
2,pharmacy,64417,3,2,2,R1620J
3,pharmacy,109229,8,3,3,R1620J
4,pharmacy,393250,3,4,4,R1620J
5,dentist,11610,-999,0,0,R1620J
6,dentist,316775,-999,1,1,R1620J
7,dentist,102409,-999,2,2,R1620J
8,dentist,104827,-999,3,3,R1620J
9,dentist,391053,-999,4,4,R1620J


In [56]:
FirstTrial = evs[evs['trial'] == 1]
pd.set_option('display.max_rows', None)
FirstTrial[['store', 'type']]

Events = evs[evs['type'] == 'pointing finished']
Words = evs[evs['type'] == 'WORDS']

# Fix the alignment issue - need to match indices or use merge/join
# This will only work if Events and Words have the same length and order
# Words['new_time'] = Events['mstime'].values  # Use .values to avoid index alignment

# Better approach - use merge or explicit index matching:
Words_new = Words.copy()  # Avoid SettingWithCopyWarning
Words_new['new_time'] = Events['mstime'].reset_index(drop=True)

In [58]:
Words_new

,eegoffset,correctPointingDirection,eegfile,efr_mark,experiment,finalrecalled,intruded,intrusion,item,itemno,...,session,stim_params,store,storeX,storeZ,subject,submittedPointingDirection,trial,type,new_time
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1654633375710
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1654633404583
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1654633443204
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1654633468645
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1654633494670
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1654633522072
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1654633549615
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1654633570027
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1654633596671
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1654633614072
